In [22]:
#подключим библиотеки
from datetime import timedelta
import cv2
import numpy as np
import os
from deepface import DeepFace

from deepface import DeepFace
import matplotlib.pyplot as plt
import time
import numpy as np
import os
import time
from PIL import Image
from tqdm import tqdm

In [12]:
#Разбиваем выдео на кадры

# то есть, если видео длительностью 30 секунд, сохраняется 10 кадров в секунду = всего сохраняется 300 кадров
SAVING_FRAMES_PER_SECOND = 10
def format_timedelta(td):
    """Служебная функция для классного форматирования объектов timedelta (например, 00:00:20.05)
    исключая микросекунды и сохраняя миллисекунды"""
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return "-" + result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"-{result}.{ms:02}".replace(":", "-")
def get_saving_frames_durations(cap, saving_fps):
    """Функция, которая возвращает список длительностей сохраняемых кадров"""
    s = []
    # получаем длительность клипа, разделив количество кадров на количество кадров в секунду
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # используем np.arange() для выполнения шагов с плавающей запятой
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s
def main(video_file):
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"
    # создаем папку по названию видео файла
    if not os.path.isdir(filename):
        os.mkdir(filename)
    # читать видео файл    
    cap = cv2.VideoCapture(video_file)
    # получить FPS видео
    fps = cap.get(cv2.CAP_PROP_FPS)
    # если наше SAVING_FRAMES_PER_SECOND больше FPS видео, то установливаем минимальное
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    # получить список длительностей кадров для сохранения
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    # начало цикла
    count = 0
    save_count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # выйти из цикла, если нет фреймов для чтения
            break
        # получаем длительность, разделив текущее количество кадров на FPS
        frame_duration = count / fps
        try:
            # получить самую первоначальную длительность для сохранения
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # список пуст, все кадры сохранены
            break
        if frame_duration >= closest_duration:
            # если ближайшая длительность меньше или равна длительности текущего кадра,
            # сохраняем фрейм
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            saveframe_name = os.path.join(filename, f"frame{frame_duration_formatted}.jpg")
            
            cv2.imwrite(saveframe_name, frame)
            save_count += 1
            #print(f"{saveframe_name} сохранён")
            # удалить текущую длительность из списка, так как этот кадр уже сохранен
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # увеличить счечик кадров count
        count += 1
        
    print(f"Итого сохранено кадров {save_count}")
    


In [13]:
main('Z:/2021/1new_dataset/1.mp4')

Итого сохранено кадров 254


In [14]:
#создаем словарь файлов в папке
def find_path(root_path):
    n = 0;
    pul = {}
    os.chdir(root_path) #проход по всем файлам в каталоге
    for root, dirs, files in tqdm(os.walk(".", topdown = False)):
        for name in files:
            paath = root_path + '/' + name
            #print (paath)
            pul[n] = paath
            n +=1
    #print(pul)
    return pul
   
    

In [15]:
# функциисоздания триплетов
faceCascade= cv2.CascadeClassifier("Z:/2021/1new_dataset/haarcascade_frontalface_default.xml")
def mix_frame(path1,path2,path3,save_path):
    """конкатинирует 3 кадра в одну картинку, вырезает область вокруг лица, сораняет результат"""
    
    img = cv2.imread(path1)
    img1 = cv2.imread(path2)
    img2 = cv2.imread(path3)

    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgGray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    imgGray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
 
    faces = faceCascade.detectMultiScale(imgGray,1.1,4)
    if len(faces) > 0:
        height, width = img.shape[:2]
 
        for (x,y,w,h) in faces:
            dw = round(w/2)
            dh = round(h/2)
            if x > dw:
                xx = x-dw
            else:
                xx = 0
            if y > dh:
                yy = y-dh
            else:
                yy = 0
       
            xxx = x+w+dw
            if xxx > width-1:
                xxx = width-1 
            yyy = y+h+dh
            if yyy > height-1:
                yyy = height-1
            
            cv2.rectangle(img,(xx,yy),(xxx,yyy),(255,0,0),2)
    
        img[:,:,0] = imgGray
        img[:,:,1] = imgGray1
        img[:,:,2] = imgGray2
    #cv2.rectangle(img,(xx,yy),(xxx,yyy),(255,0,0),2)
        Area = img[yy:yyy, xx:xxx]
        cv2.imwrite(save_path, Area)
        return Area
    else:
        return None

def mix_frame0(path1,path2,path3,save_path):
    """конкатинирует 3 кадра в одну картинку, не ищет лицо"""
    
    img = cv2.imread(path1)
    img1 = cv2.imread(path2)
    img2 = cv2.imread(path3)

    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    imgGray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    imgGray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
 
    
    img[:,:,0] = imgGray
    img[:,:,1] = imgGray1
    img[:,:,2] = imgGray2
    
    cv2.imwrite(save_path, img)
    return img
   
    

In [20]:
#папка с исходными картинками
ppp = find_path('Z:/2021/1new_dataset/1-opencv')

1it [00:00, 111.10it/s]


In [21]:
# прходим по папке, создаем триплеты
len_dict = len(ppp) - 2
save_path, _ = os.path.split(ppp[0])
save_path +='/0/'
if  not os.path.exists(save_path):
    os.mkdir(save_path)

#save_path = 'Z:/2021/1new_dataset/3/'
for n in ppp:
    #print(len_dict)
    if n < len_dict:
        path1 = ppp[n]
        path2 = ppp[n+1]
        path3 = ppp[n+2]
        mix_frame0(path1,path2,path3,save_path+str(n)+'.png')
        #mix_frame0(path1,path1,path1,save_path+str(n)+'.png')

In [ ]:
#Конвертация датасета WIDER FACE в формат tfrecord
#в папке назначения функция ожидает найти паку  images и исходными изображениями и файл  label.txt с метками


import os
import tqdm
import random
import tensorflow as tf
import numpy as np


def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def make_example(img_name, img_path, target, is_binary):
    # Create a dictionary with features that may be relevant.
    feature = {'image/img_name': _bytes_feature([img_name]),
               'image/object/bbox/xmin': _float_feature(target[:, 0]),
               'image/object/bbox/ymin': _float_feature(target[:, 1]),
               'image/object/bbox/xmax': _float_feature(target[:, 2]),
               'image/object/bbox/ymax': _float_feature(target[:, 3]),
               'image/object/landmark0/x': _float_feature(target[:, 4]),
               'image/object/landmark0/y': _float_feature(target[:, 5]),
               'image/object/landmark1/x': _float_feature(target[:, 6]),
               'image/object/landmark1/y': _float_feature(target[:, 7]),
               'image/object/landmark2/x': _float_feature(target[:, 8]),
               'image/object/landmark2/y': _float_feature(target[:, 9]),
               'image/object/landmark3/x': _float_feature(target[:, 10]),
               'image/object/landmark3/y': _float_feature(target[:, 11]),
               'image/object/landmark4/x': _float_feature(target[:, 12]),
               'image/object/landmark4/y': _float_feature(target[:, 13]),
               'image/object/landmark/valid': _float_feature(target[:, 14])}
    if is_binary:
        img_str = open(img_path, 'rb').read()
        feature['image/encoded'] = _bytes_feature([img_str])
    else:
        feature['image/img_path'] = _bytes_feature([img_path])

    return tf.train.Example(features=tf.train.Features(feature=feature))


def load_info(txt_path):
    """load info from txt"""
    img_paths = []
    words = []

    f = open(txt_path, 'r')
    lines = f.readlines()
    isFirst = True
    labels = []
    for line in lines:
        line = line.rstrip()
        if line.startswith('#'):
            if isFirst is True:
                isFirst = False
            else:
                labels_copy = labels.copy()
                words.append(labels_copy)
                labels.clear()
            path = line[2:]
            path = txt_path.replace('label.txt', 'images/') + path
            img_paths.append(path)
        else:
            line = line.split(' ')
            label = [float(x) for x in line]
            labels.append(label)

    words.append(labels)
    return img_paths, words


def get_target(labels):
    annotations = np.zeros((0, 15))
    if len(labels) == 0:
        return annotations
    for idx, label in enumerate(labels):
        annotation = np.zeros((1, 15))
        # bbox
        annotation[0, 0] = label[0]  # x1
        annotation[0, 1] = label[1]  # y1
        annotation[0, 2] = label[0] + label[2]  # x2
        annotation[0, 3] = label[1] + label[3]  # y2

        # landmarks
        annotation[0, 4] = label[4]    # l0_x
        annotation[0, 5] = label[5]    # l0_y
        annotation[0, 6] = label[7]    # l1_x
        annotation[0, 7] = label[8]    # l1_y
        annotation[0, 8] = label[10]   # l2_x
        annotation[0, 9] = label[11]   # l2_y
        annotation[0, 10] = label[13]  # l3_x
        annotation[0, 11] = label[14]  # l3_y
        annotation[0, 12] = label[16]  # l4_x
        annotation[0, 13] = label[17]  # l4_y
        if (annotation[0, 4] < 0):
            annotation[0, 14] = -1  # w/o landmark
        else:
            annotation[0, 14] = 1

        annotations = np.append(annotations, annotation, axis=0)
    target = np.array(annotations)

    return target


def to_tfrecord():
    dataset_path = './data/widerface/train'
    output_path = './data/widerface_train_bin.tfrecord'

    if not os.path.isdir(dataset_path):
        logging.info('Please define valid dataset path.')
    else:
        logging.info('Loading {}'.format(dataset_path))

    logging.info('Reading data list...')
    img_paths, words = load_info(os.path.join(dataset_path, 'label.txt'))
    samples = list(zip(img_paths, words))
    random.shuffle(samples)

    if os.path.exists(output_path):
        logging.info('{:s} already exists. Exit...'.format(
            output_path))
        exit()

    logging.info('Writing {} sample to tfrecord file...'.format(len(samples)))
    with tf.io.TFRecordWriter(output_path) as writer:
        for img_path, word in tqdm.tqdm(samples):
            target = get_target(word)
            img_name = os.path.basename(img_path).replace('.jpg', '')

            tf_example = make_example(img_name=str.encode(img_name),
                                      img_path=str.encode(img_path),
                                      target=target,
                                      is_binary=True)

            writer.write(tf_example.SerializeToString())


NameError: name 'img' is not defined

In [ ]:
#test
cv2.imshow('Color all',img)
cv2.imshow('Color blue',img[:,:,0])
cv2.imshow('Color green',img[:,:,1])
cv2.imshow('Color red',img[:,:,2])
cv2.waitKey(0)

In [23]:
from mtcnn import MTCNN

In [31]:
image = cv2.cvtColor(cv2.imread("Z:/2021/1new_dataset/6.png"), cv2.COLOR_BGR2RGB)
detector = MTCNN()
result = detector.detect_faces(image)

In [29]:

# Result is an array with all the bounding boxes detected. We know that for 'ivan.jpg' there is only one.
bounding_box = result[0]['box']
keypoints = result[0]['keypoints']

cv2.rectangle(image,
              (bounding_box[0], bounding_box[1]),
              (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
              (0,155,255),
              2)

cv2.circle(image,(keypoints['left_eye']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['right_eye']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['nose']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['mouth_left']), 2, (0,155,255), 2)
cv2.circle(image,(keypoints['mouth_right']), 2, (0,155,255), 2)

cv2.imwrite("ivan_drawn.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

True

In [33]:
len(result)

0